In [1]:
import re
from pathlib import Path
import pandas as pd
import string
import numpy as np
import ast

In [2]:
def extract_mcadl(letter_path):
    """
    Extracts relevant details from a letter of type MCADL. 
    The function looks for prisoner number, hearing date, and decision details within the file.

    Args:
    - letter_path (str): Path to the letter file.

    Returns:
    - dict: Dictionary containing extracted details (letter_id, prisoner_id, hearing date, and decision).
    """
    # Extract letter name (without extension)
    letter_name = Path(letter_path).stem

    # Open and read the letter content
    with open(letter_path, 'r', encoding='utf8') as letter:
        letter_content = letter.read().lower()  # Process content in lowercase for easier regex matching

    # Extract prison ID
    prison_id_match = re.search(r'number:(.*)\n', letter_content)
    prison_id = prison_id_match.group(1).strip() if prison_id_match else 'error'

    # Remove punctuation from prison ID
    prison_id_punc = re.sub(r'[' + re.escape(string.punctuation) + ']', ' ', prison_id)

    # Replace multiple spaces with a single space
    prison_id_space = re.sub(r'\s+', ' ', prison_id_punc)

    # Split the cleaned ID into a list and remove unwanted terms
    prison_ids_list = prison_id_space.split()
    ignore_list = ['scottish', 'prison', 'number']
    prison_ids_list_final = [id for id in prison_ids_list if id not in ignore_list]

    # Extract hearing date
    hearing_date_match = re.search(r'date:(.*)\n', letter_content)
    hearing_date = hearing_date_match.group(1).strip() if hearing_date_match else 'error'

    # Normalise date by replacing different separators with '/'
    hearing_date_sep = re.sub(r'\s+', '', hearing_date).replace('-', '/').replace('.', '/')

    # Validate and standardise date format
    hearing_date_final = hearing_date_sep if re.match(r'\b\d{2}/\d{2}/\d{4}\b', hearing_date_sep) else 'error'

    # Extract decision
    decision_match = re.search(r'decision\s*:(.*)\n', letter_content)
    decision_final = decision_match.group(1).strip() if decision_match else 'error'

    # Return extracted data as a dictionary
    return {
        'letter_id': letter_name,
        'prisoner_id': prison_ids_list_final,
        'date': hearing_date_final,
        'decision': decision_final
    }

In [3]:
def extract_ohdl(letter_path):
    """
    Extract relevant details from a letter file, including prisoner ID(s) and hearing date(s).
    
    Args:
    - letter_path (str): Path to the letter file.
    
    Returns:
    - dict: A dictionary containing the letter name, a list of prisoner IDs, and a list of hearing dates.
    """
    
    # Save letter name (without extension)
    letter_name = Path(letter_path).stem

    # Open the letter and read its content
    with open(letter_path, 'r', encoding='utf8') as letter:
        letter_content = letter.read().lower()  # Read and convert content to lowercase for easier searching
    
    # Extract prison ID
    prison_id_match = re.search(r'number:(.*)\n', letter_content)
    prison_id = prison_id_match.group(1).strip() if prison_id_match else 'error'
    
    # Clean prison ID: remove punctuation and normalize whitespace
    prison_id_punc = re.sub(r'[' + re.escape(string.punctuation) + ']', ' ', prison_id)  # Remove punctuation
    prison_id_space = re.sub(r'\s+', ' ', prison_id_punc).strip()  # Normalise whitespace
    
    # Convert cleaned prison IDs to a list of IDs (split by spaces)
    prison_ids_list = prison_id_space.split()
    
    # Extract hearing date(s)
    months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    months_pattern = '|'.join(months)
    
    # Try different patterns to find a hearing date
    hearing_date_match_1 = re.search(r'panel dates?:?(.*)\n', letter_content)
    hearing_date_match_2 = re.search(r'dates? of (panel|hearing):?(.*)\n', letter_content)
    hearing_date_match_3 = re.findall(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', letter_content)
    hearing_date_match_4 = re.findall(r'(\d{1,2}(?:st|nd|rd|th)? (' + months_pattern + ') \d{2,4})', letter_content)

    # Extract hearing date based on the matched pattern
    if hearing_date_match_1:
        hearing_date = hearing_date_match_1.group(1).strip()
    elif hearing_date_match_2:
        hearing_date = hearing_date_match_2.group(2).strip()
    elif hearing_date_match_3:
        hearing_date = hearing_date_match_3[0]  # Take the first matching date
    elif hearing_date_match_4:
        hearing_date = hearing_date_match_4[0][0]  # Take the first matching "dd month yyyy" pattern
    else:
        hearing_date = 'error'

    # Standardize hearing date format
    # Replace different separators with '/' for consistency
    hearing_date_sep = hearing_date.replace('-', '/').replace('.', '/').replace('and', ' ').replace('&', ' ').strip()

    # Convert "dd month yyyy" format to "dd/mm/yyyy" using datetime conversion
    if re.match(r'(\d{1,2}(?:st|nd|rd|th)? (' + months_pattern + ') \d{2,4})', hearing_date_sep):
        hearing_date_sep = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', hearing_date_sep)  # Remove day suffixes
        try:
            hearing_date_sep = pd.to_datetime(hearing_date_sep, format='%d %B %Y')  # Convert to datetime object
            hearing_date_sep = hearing_date_sep.strftime('%d/%m/%Y')  # Format date as dd/mm/yyyy
        except ValueError:
            pass  # In case the date format isn't valid, keep it as-is

    # If still in dd/mm/yy or similar, normalise it
    if re.match(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', hearing_date_sep):
        hearing_date_list = []
        hearing_date_check_1 = re.findall(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', hearing_date_sep)
        for hearing_date in hearing_date_check_1:
            hearing_date_parts = hearing_date.split('/')
            # Ensure day and month are 2 digits and year is 4 digits
            day = hearing_date_parts[0].zfill(2)
            month = hearing_date_parts[1].zfill(2)
            year = '20' + hearing_date_parts[2] if len(hearing_date_parts[2]) == 2 else hearing_date_parts[2]
            hearing_date_normal = f'{day}/{month}/{year}'
            hearing_date_list.append(hearing_date_normal)
    else:
        hearing_date_list = hearing_date_sep

    # Handle multiple hearing dates
    # If hearing_date_sep is a string with multiple dates (e.g., "12 June 2022 and 14 June 2022"), extract and clean all dates
    if isinstance(hearing_date_list, str):
        hearing_date_check_2 = re.findall(r'(\d{1,2}(?:st|nd|rd|th)? (' + months_pattern + ') \d{2,4})', hearing_date_sep)
        if hearing_date_check_2:
            hearing_dates_list = []
            for hearing_date in hearing_date_check_2:
                hearing_date_suffix = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', hearing_date[0])  # Remove suffixes
                hearing_date_parsed = pd.to_datetime(hearing_date_suffix, format='%d %B %Y')
                hearing_date_format = hearing_date_parsed.strftime('%d/%m/%Y')
                hearing_dates_list.append(hearing_date_format)
        else:
            hearing_dates_list = hearing_date_list
    else:
        hearing_dates_list = hearing_date_list

    # Return the extracted data in a structured dictionary
    return {
        'letter_id': letter_name, 
        'prisoner_id': prison_ids_list, 
        'date': hearing_dates_list
    }


In [4]:
def run_extraction(folder_path_segment, simplified_data, letter_type):
    """
    Extracts data from a set of letters (MCADL or OHDL) and merges it with an existing dataframe.

    Args:
    - folder_path_segment (str): Path to the folder containing letter files.
    - simplified_data (pd.DataFrame): Dataframe with additional details to merge with extracted data.
    - letter_type (str): Type of letter to extract data from ('mcadl' or 'ohdl').

    Returns:
    - pd.DataFrame: Merged dataframe with extracted details and additional data.
    """
    extract_results_list = []  # List to store extracted data
    letter_paths = list(Path(folder_path_segment).glob('*.txt*'))  # Gather all text files from the folder

    # Process each letter based on type and extract data
    for letter_path in letter_paths:
        if letter_type == 'mca':
            extract_result = extract_mcadl(letter_path)
        elif letter_type == 'oh':
            extract_result = extract_ohdl(letter_path)
        extract_results_list.append(extract_result)

    # Convert extracted data into a dataframe
    extract_results_data = pd.DataFrame.from_records(extract_results_list)

    # Merge extracted data with the provided simplified dataframe
    extract_data = pd.merge(extract_results_data, simplified_data, on='letter_id', how='inner')

    return extract_data

In [5]:
# Mapping for representation status
rep_dict = {
    'Not Represented': 'valid',
    'Represented': 'valid'
}

# Mapping for hearing types
hearing_type_dict = {
    'MCA Paper Hearing': 'mca',
    'Paper - Single Member': 'mca',
    'Oral Hearing': 'oh',
    'OH to Conclude on Papers': 'oh',
    'Duty Member Oral Hearing Request': 'oh'
}

# Mapping for decision descriptions
decision_dict = {
    'release at a future date': 'direction for release',
    'directed to oral hearing': 'directed to oral hearing',
    'remain in custody (knockback)(so) [*]': 'no direction for release',
    'open condition (so) [*]': 'open conditions',
    'no recommendation for open': 'no open conditions',
    'paper deferred': 'deferral/adjounment',
    'paper adjourned': 'deferral/adjounment',
    'release (so) [**]': 'direction for release',
    'not granted': 'no direction for release',
    'cancelled - sed passed': 'cancellation/withdrawal',
    'oh adjourned': 'deferral/adjounment',
    'oh deferral': 'deferral/adjounment',
    'withdrawn by ppcs - executive release': 'cancellation/withdrawal',
    'oral hearing - parole board (so) [*]': 'direct to oh',
    'recommend open conditions': 'open conditions',
    'no direction for release - deferral': 'no direction for release', 
    'immediate release. oral hearing not granted': 'direction for release',
    'no direction x8 x9': 'no direction for release',
    'recommend open conditions': 'open conditions',
    'no recommendation for open conditions': 'no open conditions',
    'release (so) [*]': 'direction for release',
    'direct release': 'direction for release',
    'adjourned - on the day': 'deferral/adjounment',
    'oh to conclude on papers': 'direct to oh',
    'directed to oral hearing': 'direct to oh',
    'recommend release': 'direction for release',
    'adjourned - before the hearing': 'deferral/adjounment',
    'sent to a directions hearings': 'direct to oh',
    'proceeding to a directions hearing': 'direct to oh',
    'withdrawn by ppcs - mh case': 'cancellation/withdrawal',
    'deferred': 'deferral/adjounment',
    'sent to multi member panel': 'direct to oh',
    'withdrawn by ppcs - deportation': 'cancellation/withdrawal',
    'withdrawn by ppcs - death in custody': 'cancellation/withdrawal',
    'no hearing sed within 20 weeks': 'cancellation/withdrawal',
    'sent back to mca to conclude on papers': 'cancellation/withdrawal',
    'other': np.nan,
    'not specified': np.nan,
    'not applicable': np.nan
}

# Mapping for decision validity
decision_valid_dict = {
    'no direction for release': 'valid',
    'direction for release': 'valid',
    'open conditions': 'valid',
    'no open conditions': 'valid',
    'direct to oh': 'not valid',
    'deferral/adjounment': 'not valid',
    'cancellation/withdrawal': 'not valid'
}

In [6]:
def split_prisoner_id(prisoner_id_str):
    """
    Splits a prisoner ID string into individual components, removing any whitespace or punctuation.

    Args:
    - prisoner_id_str (str): The prisoner ID string to split.

    Returns:
    - list: A list of prisoner ID components after splitting and cleaning.
            If input is not a string, returns 'error'.
    """
    
    # Compile a regex pattern to match various delimiters (/, (), spaces, and -)
    pattern = re.compile(r'\s*(?:/|\(|\)|\s+|-)s*')

    # Ensure the input is a string before processing
    if isinstance(prisoner_id_str, str):
        # Split the string based on the pattern and filter out empty strings
        split_string = pattern.split(prisoner_id_str)
        clean_string = [s for s in split_string if s.strip()]
    else:
        clean_string = 'error'
    
    return clean_string

In [7]:
def duplicate_filtering(dupe_data):
    """
    Filters duplicates from the input DataFrame based on decision and representation validity.

    Args:
    - dupe_data (pd.DataFrame): DataFrame containing duplicates with 'prisoner_id', 'decision', and 'representation_status_description'.

    Returns:
    - pd.DataFrame: Filtered DataFrame with duplicates resolved.
    """
    
    # Create a copy of the DataFrame to avoid modifying the original
    dupe_copy = dupe_data.copy()

    # Add a new column 'rep_valid' to indicate if the representation status is valid
    dupe_copy['rep_valid'] = dupe_copy['representation_status_description'].replace(rep_dict)

    # Get the unique prisoner IDs from the DataFrame
    prisoner_ids = set(dupe_copy['prisoner_id'].unique())

    # Create an empty DataFrame to store the final filtered results
    relevant_data = pd.DataFrame(columns=dupe_copy.columns)
    
    # Iterate over each prisoner ID and filter rows based on decision and representation status
    for prisoner_id in prisoner_ids:
        
        subset_data = dupe_copy[dupe_copy['prisoner_id'] == prisoner_id]
        subset_data_decisions = set(subset_data['decision_valid'])
        subset_data_rep = set(subset_data['rep_valid'])
        
        # Multiple decision statuses present
        if len(subset_data_decisions) != 1:
            
            # Check if any decision is NaN
            if subset_data[subset_data['decision'].isna()].empty:
                
                # Prioritize valid decisions
                if 'valid' in subset_data_decisions:
                    relevant_row = subset_data[subset_data['decision_valid'] == 'valid']

                # Handle mixed representation statuses
                elif len(subset_data_rep) != 1 and 'valid' in subset_data_rep:
                    relevant_row = subset_data[subset_data['rep_valid'] == 'valid']
                    
            else:
                # If no valid decision found, filter by non-NaN decisions
                relevant_row = subset_data[subset_data['decision'].notna()]
        
        else:
            # If there's only one decision status, filter based on representation validity
            if len(subset_data_rep) != 1 and 'valid' in subset_data_rep:
                relevant_row = subset_data[subset_data['rep_valid'] == 'valid']
        
        # Concatenate the filtered rows into the final DataFrame
        relevant_data = pd.concat([relevant_data, relevant_row])

    # Drop the temporary 'rep_valid' column
    return relevant_data.drop(['rep_valid'], axis=1)

In [8]:
def prepare_supp(supp_data):
    """
    Prepares supplementary hearing data by cleaning, transforming, and removing duplicates.

    Args:
    - supp_data (pd.DataFrame): The input DataFrame containing supplementary hearing data.

    Returns:
    - pd.DataFrame: The cleaned and prepared DataFrame with duplicates removed.
    """
    
    # Create copies to avoid modifying original data
    supp_copy = supp_data.copy()

    # Data transformation and cleaning
    supp_copy['prisoner_id'] = supp_copy['PRISON_NUMBER'].str.lower()
    supp_copy['hearing_date'] = pd.to_datetime(supp_copy['HEARING_START_DATE'])
    supp_copy['hearing_type'] = supp_copy['HEARING_TYPE_DESCRIPTION'].replace(hearing_type_dict)
    supp_copy['decision'] = supp_copy['HEARING_RESULT_DESCRIPTION'].str.lower().replace(decision_dict)
    supp_copy['decision_valid'] = supp_copy['decision'].replace(decision_valid_dict)

    # Rename columns to lowercase and select relevant columns
    supp_copy.columns = supp_copy.columns.str.lower()
    supp_copy = supp_copy[['prisoner_id', 'hearing_date', 'current_establishment_description', 'custody_type_description', 
                           'ethnicity_description', 'gender', 'decision', 'decision_valid', 'nationality_description',
                           'original_target_date', 'review_reason_description', 'hearing_type', 'representation_status_description',
                           'years_old_at_hearing']]
    
    # Remove duplicates
    supp_copy = supp_copy.drop_duplicates(keep='first').reset_index(drop=True)

    # Apply prisoner ID splitting and expand rows for multiple IDs
    supp_copy['prisoner_id'] = supp_copy['prisoner_id'].apply(split_prisoner_id)
    supp_copy_long = supp_copy.explode('prisoner_id')
    supp_copy_long = supp_copy_long[supp_copy_long['prisoner_id'] != 'error'].reset_index(drop=True)

    # Handle duplicates
    supp_copy_duplicates = supp_copy_long[supp_copy_long.duplicated(subset=('prisoner_id', 'hearing_date'), keep=False)]
    supp_copy_no_duplicates = supp_copy_long.drop_duplicates(subset=('prisoner_id', 'hearing_date'), keep=False)
    supp_copy_correct_duplicates = duplicate_filtering(supp_copy_duplicates)
    prepared_supp_data = pd.concat([supp_copy_no_duplicates, supp_copy_correct_duplicates]).reset_index(drop=True)

    return prepared_supp_data

In [9]:
def prepare_extract_mcadl(extract_data_mcadl, prepared_supp_data):
    """
    Prepares the extracted MCA hearing data by mapping decisions, formatting dates, and filtering relevant prisoner records.

    Args:
    - extract_data_mcadl (pd.DataFrame): The extracted dataset containing prisoner hearing information.
    - prepared_supp_data (pd.DataFrame): The supplementary dataset with pre-processed MCA hearing data.

    Returns:
    - pd.DataFrame: The prepared extract data.
    """
    
    # Filter the supplementary data for MCA hearings
    prepared_supp_data_mcadl = prepared_supp_data[prepared_supp_data['hearing_type'] == 'mca']
    prepared_supp_data_prisoner_ids = set(prepared_supp_data_mcadl['prisoner_id'])  # Set of prisoner IDs in supplementary MCA data

    # Create a copy of the extract data to avoid modifying the original
    extract_copy_mcadl = extract_data_mcadl.copy()

    # Remove rows with invalid date values and convert dates to datetime format
    extract_copy_mcadl = extract_copy_mcadl[extract_copy_mcadl['date'] != 'error'].reset_index(drop=True)
    extract_copy_mcadl['hearing_date'] = pd.to_datetime(extract_copy_mcadl['date'], format='%d/%m/%Y')
    extract_copy_mcadl = extract_copy_mcadl.drop('date', axis=1)  # Drop original 'date' column after conversion

    # Explode any lists in the 'prisoner_id' column into separate rows
    extract_copy_long_mcadl = extract_copy_mcadl.explode('prisoner_id').reset_index(drop=True)

    # Map the decision descriptions to standardized values
    extract_copy_long_mcadl['decision'] = extract_copy_long_mcadl['decision'].str.lower().replace(decision_dict)
    # Add a column to indicate if the decision is valid
    extract_copy_long_mcadl['decision_valid'] = extract_copy_long_mcadl['decision'].replace(decision_valid_dict)

    # Drop duplicate rows based on 'prisoner_id' and 'hearing_date'
    extract_copy_long_mcadl = extract_copy_long_mcadl.drop_duplicates(subset=('prisoner_id', 'hearing_date'), keep='first').reset_index(drop=True)

    # Get the set of prisoner IDs from the extract data
    extract_copy_prisoner_ids_mcadl = set(extract_copy_long_mcadl['prisoner_id'])

    # Find the overlapping prisoner IDs between extract data and supplementary MCA data
    overlapping_prisoner_ids = [prisoner_id for prisoner_id in extract_copy_prisoner_ids_mcadl if prisoner_id in prepared_supp_data_prisoner_ids]

    # Filter the extract data to include only relevant records with overlapping prisoner IDs
    prepared_extract_data_mcadl = extract_copy_long_mcadl[extract_copy_long_mcadl['prisoner_id'].isin(overlapping_prisoner_ids)].reset_index(drop=True)

    return prepared_extract_data_mcadl


In [10]:
def prepare_extract_ohdl(extract_data_ohdl, prepared_supp_data):
    """
    Prepares the extracted Oral Hearing (OH) data by formatting dates and filtering relevant prisoner records.

    Args:
    - extract_data_ohdl (pd.DataFrame): The extracted dataset containing prisoner hearing information.
    - prepared_supp_data (pd.DataFrame): The supplementary dataset.

    Returns:
    - pd.DataFrame: The relevant extracted data containing OH hearings.
    """
    
    # Filter the supplementary data for Oral Hearings (OH)
    prepared_supp_data_ohdl = prepared_supp_data[prepared_supp_data['hearing_type'] == 'oh']
    prepared_supp_data_prisoner_ids = set(prepared_supp_data_ohdl['prisoner_id'])  # Set of prisoner IDs in supplementary OH data

    # Create a copy of the extract data to avoid modifying the original
    extract_copy_ohdl = extract_data_ohdl.copy()

    # Drop rows with missing dates and reset the index
    extract_copy_ohdl = extract_copy_ohdl.dropna(subset=['date']).reset_index(drop=True)
    extract_copy_ohdl = extract_copy_ohdl[extract_copy_ohdl['date'] != 'error'].reset_index(drop=True)

    # Explode any lists in the 'date' column into separate rows
    extract_copy_long_date_ohdl = extract_copy_ohdl.explode('date').reset_index(drop=True)
    # Convert the date strings to datetime format
    extract_copy_long_date_ohdl['hearing_date'] = pd.to_datetime(extract_copy_long_date_ohdl['date'], format='%d/%m/%Y')
    extract_copy_long_date_ohdl = extract_copy_long_date_ohdl.drop('date', axis=1)  # Drop original 'date' column

    # Explode any lists in the 'prisoner_id' column into separate rows
    extract_copy_long_id_ohdl = extract_copy_long_date_ohdl.explode('prisoner_id').reset_index(drop=True)
    # Filter out rows where 'prisoner_id' is marked as 'error'
    extract_copy_long_id_ohdl = extract_copy_long_id_ohdl[extract_copy_long_id_ohdl['prisoner_id'] != 'error']

    # Drop duplicate rows based on 'prisoner_id' and 'hearing_date'
    extract_copy_long_id_ohdl = extract_copy_long_id_ohdl.drop_duplicates(subset=('prisoner_id', 'hearing_date'), keep='first').reset_index(drop=True)

    # Get the set of prisoner IDs from the extract data
    extract_copy_prisoner_ids_ohdl = set(extract_copy_long_id_ohdl['prisoner_id'])

    # Find the overlapping prisoner IDs between extract data and supplementary OH data
    overlapping_prisoner_ids = [prisoner_id for prisoner_id in extract_copy_prisoner_ids_ohdl if prisoner_id in prepared_supp_data_prisoner_ids]

    # Filter the extract data to include only relevant records with overlapping prisoner IDs
    prepared_extract_data_ohdl = extract_copy_long_id_ohdl[extract_copy_long_id_ohdl['prisoner_id'].isin(overlapping_prisoner_ids)].reset_index(drop=True)

    return prepared_extract_data_ohdl

In [11]:
def prepare_extract(extract_data, prepared_supp_data, letter_type):
    """
    Prepares the extracted data based on the letter type.

    This function selects the appropriate preparation method depending on 
    the given `letter_type` and processes the `extract_data`, using the `prepared_supp_data`.

    Args:
    - extract_data (pd.DataFrame): The extracted dataset containing prisoner hearing information.
    - prepared_supp_data (pd.DataFrame): The supplementary dataset.
    - letter_type (str): Type of the letter, can be either 'mca' or 'oh'.

    Returns:
    - pd.DataFrame: The relevant extracted data for either OHDLs or MCADLs.
    """
    
    # Determine preparation function based on letter type
    if letter_type == 'mca':
        # Prepare data for 'mca' letter type
        prepared_extract_data = prepare_extract_mcadl(extract_data, prepared_supp_data)
    elif letter_type == 'oh':
        # Prepare data for 'oh' letter type
        prepared_extract_data = prepare_extract_ohdl(extract_data, prepared_supp_data)
    
    return prepared_extract_data


In [12]:
def link_row(extract_data_row, prepared_supp_data):
    """
    Links a row from extract data to the relevant supplementary data based on prisoner ID and hearing date.
    
    Parameters:
    - extract_data_row (pd.Series): A single row from the extract data containing prisoner information.
    - prepared_supp_data (pd.DataFrame): The supplementary data containing relevant hearing details.
    
    Returns:
    - pd.DataFrame: A merged single row DataFrame containing information from both the extract and relevant supplemental data.
    """
    
    prison_id = extract_data_row['prisoner_id']  # Get the prisoner ID from the extract data row
    hearing_date = extract_data_row['hearing_date']  # Get the hearing date from the extract data row

    # Filter supplemental data for rows with the same prisoner ID
    supp_data_relevant = prepared_supp_data[prepared_supp_data['prisoner_id'] == prison_id].copy()
    # Calculate the absolute difference in days between the hearing dates
    supp_data_relevant['difference_days'] = (supp_data_relevant['hearing_date'] - hearing_date).dt.days.abs()

    # Check for same day with valid decisions
    if not supp_data_relevant[(supp_data_relevant['difference_days'] == 0) & (supp_data_relevant['decision_valid'] == 'valid')].empty:
        supp_data_relevant_row = supp_data_relevant[(supp_data_relevant['difference_days'] == 0) & (supp_data_relevant['decision_valid'] == 'valid')]
        link_type = 'same day, valid decision'
    
    # Check for within 1 month valid decisions
    elif not supp_data_relevant[(supp_data_relevant['difference_days'] < 31) & (supp_data_relevant['decision_valid'] == 'valid')].empty:
        supp_data_relevant_rows = supp_data_relevant[(supp_data_relevant['difference_days'] < 31) & (supp_data_relevant['decision_valid'] == 'valid')]
        supp_data_relevant_row_index = supp_data_relevant_rows['difference_days'].idxmin()
        supp_data_relevant_row = pd.DataFrame(supp_data_relevant_rows.loc[supp_data_relevant_row_index]).transpose()
        link_type = 'within 1 month, valid decision'
    
    # Check for same day with non-valid decisions
    elif not supp_data_relevant[supp_data_relevant['difference_days'] == 0].empty:
        supp_data_relevant_row = supp_data_relevant[supp_data_relevant['difference_days'] == 0]
        link_type = 'same day, non valid decision'

    # Default case: Find the closest day regardless of decision validity
    else:
        supp_data_relevant_row_index = supp_data_relevant['difference_days'].idxmin()
        supp_data_relevant_row = pd.DataFrame(supp_data_relevant.loc[supp_data_relevant_row_index]).transpose()
        
        if supp_data_relevant_row['decision_valid'][supp_data_relevant_row_index] == 'valid':
            link_type = 'closest day, valid decision'
        else:
            link_type = 'closest day, non valid decision'

    supp_data_relevant_row['link_type'] = link_type  # Add link type to the final row

    extract_data_row = pd.DataFrame(extract_data_row).transpose()  # Convert original row to DataFrame

    # Merge original row with the final relevant row from supplemental data
    linked_row = pd.merge(extract_data_row, supp_data_relevant_row, on='prisoner_id', suffixes=('_letter', '_supp'))

    return linked_row  # Return the merged DataFrame

In [13]:
def link_mcadl(prepared_extract_data_mcadl, prepared_supp_data):
    """
    Executes the linkage process for MCADL data by iterating through prepared extract data and linking with supplemental data.
    
    Parameters:
    - prepared_extract_data_mcadl (pd.DataFrame): The extracted mcadl data prepared for linkage.
    - prepared_supp_data (pd.DataFrame): The supplementary data containing relevant hearing details.
    
    Returns:
    - pd.DataFrame: The final linked DataFrame without duplicates and irrelevant link types.
    """
    
    # Concatenate all linked rows by processing each row in prepared extract data
    linked_rows = pd.concat([link_row(prepared_extract_data_mcadl.loc[i], prepared_supp_data) for i in range(len(prepared_extract_data_mcadl))]).reset_index(drop=True)
    
    # Remove duplicates from linked rows based on 'letter_id'
    linked_rows_no_duplicates = linked_rows.drop_duplicates(subset=('letter_id'), keep=False)
    linked_rows_columns = linked_rows.columns.difference(['prisoner_id'])  # Identify duplicate columns
    linked_row_duplicates = linked_rows[linked_rows.duplicated(subset=linked_rows_columns, keep='first')]  # Find duplicate letter IDs

    # Concatenate linked rows without duplicates with duplicated rows
    linked_rows_relevant = pd.concat([linked_rows_no_duplicates, linked_row_duplicates]).reset_index(drop=True)

    # Define link types to exclude from final data
    link_types_exclude = ['closest day, non valid decision', 'same day, non valid decision', 'closest day, valid decision']

    # Filter out excluded link types from the final data
    linked_data = linked_rows_relevant.drop(linked_rows_relevant[linked_rows_relevant['link_type'].isin(link_types_exclude)].index).reset_index(drop=True)
    # Drop unnecessary columns from final data
    linked_data = linked_data.drop(['decision_letter', 'decision_valid_letter', 'hearing_date_supp', 'decision_valid_supp'], axis=1)
    # Rename decision and date columns
    linked_data.rename(columns={'decision_supp': 'decision', 'hearing_date_letter': 'hearing_date'}, inplace=True)

    return linked_data  # Return the final linked DataFrame

In [14]:
def match_idx(relevant_rows):
    """
    Matches relevant rows based on various rules to determine the best row for linkage.
    
    Parameters:
    - relevant_rows (pd.DataFrame): A DataFrame containing potential matching relevant rows.
    
    Returns:
    - tuple: The index of the chosen candidate and the applied rule.
    """
    
    relevant_rows_copy = relevant_rows.copy()
    relevant_rows_copy['difference_days'] = relevant_rows_copy['difference_days'].astype(int)  # Ensure difference_days is integer

    # Check uniqueness of prisoner ID, letter date, and supplemental date
    prisoner_id_uniq = relevant_rows_copy['prisoner_id'].nunique() == 1
    letter_date_uniq = relevant_rows_copy['hearing_date_letter'].nunique() == 1
    supp_date_uniq = relevant_rows_copy['hearing_date_supp'].nunique() == 1

    valid_mask = relevant_rows_copy['decision_valid'] == 'valid'  # Mask for valid decisions
    n_valid = valid_mask.sum()  # Count of valid decisions

    chosen_row_idx = None  # Initialize chosen index
    applied_rule = None  # Initialize applied rule

    # Rule 1: PID same, LD diff, SD diff
    if prisoner_id_uniq and not letter_date_uniq and not supp_date_uniq:
        applied_rule = '1'
        if n_valid == 0:
            # Choose the latest date if no valid decisions
            chosen_row_idx = relevant_rows_copy['hearing_date_ohdl'].idxmax()
        elif n_valid == 1:
            # Choose the single valid candidate
            chosen_row_idx = relevant_rows_copy[valid_mask].index[0]
        else:
            # Choose the candidate with the lowest difference in days
            chosen_row_idx = relevant_rows_copy['difference_days'].idxmin()

    # Rule 2: PID same, LD diff, SD same
    elif prisoner_id_uniq and not letter_date_uniq and supp_date_uniq:
        chosen_row_idx = relevant_rows_copy['difference_days'].idxmin()  # Choose the lowest difference
        applied_rule = '2'

    # Rule 3: PID diff, LD same, SD diff
    elif not prisoner_id_uniq and letter_date_uniq and not supp_date_uniq:
        chosen_row_idx = relevant_rows_copy['difference_days'].idxmin()  # Choose the lowest difference
        applied_rule = '3'

    # Rule 4: PID diff, LD same, SD same
    elif not prisoner_id_uniq and letter_date_uniq and supp_date_uniq:
        chosen_row_idx = relevant_rows_copy.index[0]  # Choose the first index
        applied_rule = '4'

    # Rule 5: PID diff, LD diff, SD same
    elif (not prisoner_id_uniq and not letter_date_uniq and not supp_date_uniq) or (not prisoner_id_uniq and not letter_date_uniq and supp_date_uniq):
        applied_rule = '5'
        if n_valid > 1 and relevant_rows_copy['difference_days'].nunique() == 1:
            chosen_row_idx = relevant_rows_copy.index[0]  # Choose the first index
        elif n_valid > 1:
            chosen_row_idx = relevant_rows_copy['difference_days'].idxmin()  # Choose the lowest difference
        elif n_valid == 1:
            chosen_row_idx = relevant_rows_copy[valid_mask].index[0]  # Choose the valid candidate

    if chosen_row_idx is None:
        print(relevant_rows_copy)  # Print relevant rows if no choice is made

    return chosen_row_idx, applied_rule  # Return chosen index and applied rule

In [15]:
def link_ohdl(prepared_extract_data_ohdl, prepared_supp_data):
    """
    Executes the linkage process for OHLD data by iterating through prepared extract data and linking with supplemental data.
    
    Parameters:
    - prepared_extract_data_ohdl (pd.DataFrame): The extracted data prepared for linkage.
    - prepared_supp_data (pd.DataFrame): The supplemental data containing relevant hearing details.
    
    Returns:
    - pd.DataFrame: The final linked DataFrame without duplicates and irrelevant link types.
    """
    
    # Concatenate all linked rows by processing each row in prepared extract data
    all_linked_rows = pd.concat([link_row(prepared_extract_data_ohdl.loc[i], prepared_supp_data) for i in range(len(prepared_extract_data_ohdl))]).reset_index(drop=True)

    # Remove duplicates from linked rows based on 'letter_id'
    linked_rows_no_duplicates = all_linked_rows.drop_duplicates(subset=('letter_id'), keep=False)
    linked_rows_duplicates = all_linked_rows[all_linked_rows.duplicated('letter_id', keep=False)]  # Identify duplicated rows

    chosen_row_idxs_list = []  # List to hold chosen indices
    applied_rules_list = []  # List to hold applied rules

    # Process each unique letter ID for duplicated rows
    for letter_id in linked_rows_duplicates['letter_id'].unique():
        linked_rows_duplicates_all = linked_rows_duplicates[linked_rows_duplicates['letter_id'] == letter_id]
        chosen_row_idx, applied_rule = match_idx(linked_rows_duplicates_all)  # Find the best candidate
        chosen_row_idxs_list.append(chosen_row_idx)  # Store chosen index
        applied_rules_list.append(applied_rule)  # Store applied rule

    # Create relevant DataFrame for duplicated rows with applied rules
    linked_rows_duplicates_relevant = linked_rows_duplicates.loc[chosen_row_idxs_list].copy()
    linked_rows_duplicates_relevant['applied_rule'] = applied_rules_list

    # Combine linked rows without duplicates with relevant duplicated rows
    linked_rows_relevant = pd.concat([linked_rows_no_duplicates, linked_rows_duplicates_relevant.drop('applied_rule', axis=1)]).reset_index(drop=True)

    # Define link types to exclude from final data
    link_types_exclude = ['closest day, non valid decision', 'same day, non valid decision', 'closest day, valid decision']

    # Filter out excluded link types from the final data
    linked_data = linked_rows_relevant.drop(linked_rows_relevant[linked_rows_relevant['link_type'].isin(link_types_exclude)].index).reset_index(drop=True)
    # Drop unnecessary columns from final data
    linked_data = linked_data.drop(['decision_valid', 'hearing_date_supp'], axis=1)
    # Rename date column
    linked_data.rename(columns={'hearing_date_letter': 'hearing_date'}, inplace=True)

    return linked_data  # Return the final linked DataFrame

In [16]:
def run_linkage(prepared_extract_data, prepared_supp_data, letter_type):
    """
    Links the prepared extract data with the supplementary data based on the letter type.

    This function runs the appropriate linkage process depending on the given `letter_type`.

    Args:
    - prepared_extract_data (pd.DataFrame): The extracted data prepared for linkage.
    - prepared_supp_data (pd.DataFrame): The supplementary data containing relevant hearing details.
    - letter_type (str): Type of the letter, either 'mca' or 'oh'.

    Returns:
    - pd.DataFrame: The final linked DataFrame without duplicates and irrelevant link types.
    """
    
    # Determine linkage function based on letter type
    if letter_type == 'mca':
        # Link data for 'mca' letter type
        linked_data = link_mcadl(prepared_extract_data, prepared_supp_data)
    elif letter_type == 'oh':
        # Link data for 'oh' letter type
        linked_data = link_ohdl(prepared_extract_data, prepared_supp_data)
    
    return linked_data

In [17]:
def run_linkage_process(folder_path_segment, simplified_data, supp_data, letter_type):
    """
    Executes the entire linkage process, including extraction, preparation, and linking.

    This function orchestrates the process by running extraction, preparing both the 
    supplementary data and extracted data, and finally linking the data together.

    Args:
    - folder_path_segment (str): The path to the folder containing the original decision letters.
    - simplified_data (pd.DataFrame): Dataframe with simplified offending history details to join with extracted data.
    - supp_data (pd.DataFrame): The input DataFrame containing supplementary hearing data.
    - letter_type (str): Type of the letter, either 'mca' or 'oh'.

    Returns:
    - pd.DataFrame: The final linked data after the full linkage process.
    """
    
    # Step 1: Run the extraction process
    extract_data = run_extraction(folder_path_segment, simplified_data, letter_type)
    
    # Step 2: Prepare the supplementary data
    prepared_supp = prepare_supp(supp_data)
    
    # Step 3: Prepare the extracted data based on the letter type
    prepared_extract = prepare_extract(extract_data, prepared_supp, letter_type)
    
    # Step 4: Run the linkage process based on the letter type
    linked_data = run_linkage(prepared_extract, prepared_supp, letter_type)
    
    return linked_data

In [18]:
# Load the simplified offending history data for each letter type
simplified_mcadl = pd.read_excel('../data/primary_data/extract/mcadl/simplified_mcadl.xlsx', dtype={'letter_id': str})
simplified_ohdl = pd.read_excel('../data/primary_data/extract/ohdl/simplified_ohdl.xlsx', dtype={'letter_id': str})
# Load supp data
supp_data = pd.read_excel('../data/supplementary_data/supp_data.xlsx')

In [19]:
linked_mcadl = run_linkage_process('../data/primary_data/letters/mcadl/segmented_dls/', simplified_mcadl, supp_data, 'mca')
linked_ohdl = run_linkage_process('../data/primary_data/letters/ohdl/segmented_dls/', simplified_ohdl, supp_data, 'oh')

/var/folders/p0/kd5c1wdj7_d6n50vl898smy00000gn/T/ipykernel_79823/312419578.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  relevant_data = pd.concat([relevant_data, relevant_row])
/var/folders/p0/kd5c1wdj7_d6n50vl898smy00000gn/T/ipykernel_79823/312419578.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  relevant_data = pd.concat([relevant_data, relevant_row])
/var/folders/p0/kd5c1wdj7_d6n50vl898smy00000gn/T/ipykernel_79823/886604316.py:48: SettingWithCopyWarning: 
A value is trying to be

In [20]:
# Save linked data
linked_mcadl.to_excel('../data/linked_data/mcadl/linked_mcadl.xlsx', index=False)
linked_ohdl.to_excel('../data/linked_data/ohdl/linked_ohdl.xlsx', index=False)